In [31]:
#Requires Python 3.8+ for inverse mod using pow()

# Imports

In [32]:
import random
import secrets
from decimal import *

# Functions

Returns the GCD of 2 numbers

In [33]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Rabin-Miller Primality Test<br>
Checks if a number is prime, p is the number to be tested, s is the number of rounds<br>
Number of rounds should ensure that the probability of p not being prime is < 2^(-80) 

In [34]:
def RMPT(p, s):
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

Converts a string to ascii

In [35]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

Converts ascii back to string

In [36]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generates the decryption key using public key e and 2 prime numbers p and q

In [37]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Encrypts a number with public key e

In [38]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Encrypts a number using private key d

In [39]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, adds one to make it odd<br>
Tries to factorize the number with small primes to determine if composite<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [40]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

In [65]:
#Workaround for getting the cube root of a huge integer

def cube_root(encrypted_message):
    minprec = 27

    encrypted_message_len = len(str(encrypted_message))

    if encrypted_message_len > minprec: 
        getcontext().prec = encrypted_message_len
    else:
        getcontext().prec = minprec

    encrypted_message = Decimal(encrypted_message)
    power = Decimal(1)/Decimal(3)

    #Gets cube root of encrypted message
    decrypted_message = encrypted_message**power
    decrypted_message = int(decrypted_message.quantize(Decimal('1.'), rounding=ROUND_UP))
    
    return decrypted_message

# RSA 1024

Let's try to encrypt a message using RSA 1024

Generate the primes p q

Use encryption exponent of 3

Get decryption exponent

In [49]:
p = Generate_Prime(1024, 3)
q = Generate_Prime(1024, 3)
N = p * q

e = 3

#Rerun this cell if if GCD of e and phi N not 1 (Might take a few retries)
d = RSA_generate_D(e, p, q)

83554548638708630055736280767356681884944817077264512694443649279240132573527757309626748557829326571861172138223452534622163470029542756197959820198402167763890281962190101712418785560179025928801364851671040058631401137296475933476031647350735753732761045999155254717806349732309566083193986657428606222741
104873440792724051748038298728921401360478503541686052469900979542629873361719654747621407995123529656408294679275801098624229328307019418953593669613091822577484269399676466326935692058892495209063210704041862250238703377678452163842744860705001761800666930663424021157741278268822381019887581440779391317183


In [50]:
message = "Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3"

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

72101108108111044032116104105115032109101115115097103101032105115032115101099117114101032105110032082083065049048050052032098117116032119101097107032105110032082083065032050048052056032119105116104111117116032112097100100105110103032119105116104032097032115109097108108032101120112111110101110116032111102032051


1033

In [73]:
encrypted_message = RSA_encrypt(ascii_message, e, N)

In [74]:
print(encrypted_message)

96517040608012069739646774328123050437318010368011946522080786567928074875744398455920291662633616828227110535219751557507111903065235122909685762059484587165749911403355186635486398842434318412700060840305725321505213474041936693103779223526957041641940562201313490083631172240066045375482039709407123750235131598097126730510329089201735534060034678085216125818759163443583413072793991460252388117813177443850501164180637830569652858129581260643890987405276170252786702028832366567143396582372467230919836926410248104326486884838913179058340219911846560744889901534029678879557242486618563986592961055908541702403


Python is unable to handle such large numbers for cube root

In [77]:
decrypted_message = encrypted_message ** 1/3

OverflowError: integer division result too large for a float

Use workaround function to get cube root

In [78]:
decrypted_message = cube_root(encrypted_message)

print(convert_to_string(decrypted_message))

ɋ>ɊǜơbÓɈʟÛ*ŨÎƤŚǄ͐ń̓ÛŅŜSʫά½¤͜ƞğʇɮƳ̘̿ȳƞ̿ZʗǺΔƙ͹ɧƶÏ΢fGʖ²ƨʬūŚćʣĭkʨɧĳ


Message is still encrypted even after cube root

Lets properly decrypt the message now

In [83]:
decrypted_message = RSA_encrypt(encrypted_message, d, N)

In [84]:
print(convert_to_string(decrypted_message))

Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3


# RSA 2048

Let's try to encrypt a message using RSA 2048

Generate the primes p q

Use encryption exponent of 3 again

Get decryption exponent

In [91]:
p = Generate_Prime(2048, 3)
q = Generate_Prime(2048, 3)
N = p * q

e = 3

#Rerun this cell if if GCD of e and phi N not 1 (Might take a few retries)
d = RSA_generate_D(e, p, q)

23301205960253318774455978293729997703121977084255450361902965081783025471348430780733817148375594724884512725381914796945587914796958273213096067668797565949096022961485526206392179082163598593682039504044419299712622908788242530820519619668316424689172429500273743252218069865784744391467476222822485289785821995549987596234989734072120981409405040639764831880501221158361982778866714009038452544988500147114797583600605038386607401567959418529765790035006128718776601513971913204814979456387716909142744094763199418785057086191542201156735930726654846971983489899824221561408202833753446551385409744183731504658423
1307100250046400167457805203468786934403729081645227175958266891957872026228585346902997417149087822574085805220073496753515882730364384235262779571539374741549715346514990017686720500431610394827884401042355028032589125645759189196211559957684495499115142850531518391714240624569188703240666349438526169704626670448525107386858701479711333905875918547255762846235242680384937283634

In [92]:
message = "Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3"

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

72101108108111044032116104105115032109101115115097103101032105115032115101099117114101032105110032082083065049048050052032098117116032119101097107032105110032082083065032050048052056032119105116104111117116032112097100100105110103032119105116104032097032115109097108108032101120112111110101110116032111102032051


1033

In [93]:
encrypted_message = RSA_encrypt(ascii_message, 3, N)

In [94]:
print(encrypted_message)

374822642466453503471129234953054526212879179388354880533829447986843654790252314554033043646746764449913639182639657708742449037448950714078946330611043527252186205143428991207943778445949669261828678244029033364957051851646843200248207959737882920354740993629980412340688249208986011917884001719076791515449239894248936982703403870041158347994141996971536956348876306054595083780967728076706532035714204633008080071541961769864324054181130056979714597750740643162189705244532503479892077386565813438424295161109106397041214616802454831180487114975435436195881465431700791503705277369993863949588713707081687514834613488238651365075709738120939125370934830072994404453950653829807147232013328123932604129717807697923904573366449573757101712631775756908010422087048442049253902421391121963980754726364178279941244392279920641838047656797066648613408324642902777263968839370282113878069675238833717309324302239332461043662637827828651


In [95]:
print(convert_to_string(encrypted_message))

ň{Τɜˍ̧ʹΛΈȽŮǁȽ˵eˈɷ̇˴Ό
ƦW0ƺ1ýΆƥƇyσϔ˲˖Ŭ²ėέôƈėΘʁ͆/ʐ̝BʈɥƘńʂΆ̉ćψ͇ŲĚqͮEʣî́ˍĵńĮïŌǍ+ʖɽ̻̼ʋ


Since the number of bits of the plain text is much smaller than 2048 bits, even when the plain text raised to the power of 3 it is still smaller than 2048 bits which means the mod operation does nothing, and i can just use the cube root to find the original message

In [96]:
decrypted_message = cube_root(encrypted_message)

In [97]:
print(convert_to_string(decrypted_message))

Hello, this message is secure in RSA1024 but weak in RSA 2048 without padding with a small exponent of 3


# Conclusions

This exploit can be avoided by simply padding the original message to make it the same size as N or by choosing a larger exponent